In [1]:
from dotenv import load_dotenv
import sys

sys.path.append("../common")
load_dotenv()

True

In [2]:
import os
from langsmith_tracker import set_tracking
from langchain_print import stream_response

# 인스턴스를 생성할 때 필요한 매개변수를 전달합니다.
set_tracking(project_name="04.Models")

Langsmith 추적이 활성화되었습니다. [프로젝트명: 04.Models]


### 직렬화(Serialization)

모델을 저장 가능한 형식으로 변환화는 가정을 말합니다. 우리가 만든 체인이나 프롬프트들을 파일형태도 저장할때 컨버트 하는 방법 중에 하나입니다.
  
직렬화 목적
- 모델 재사용
- 모델 배포 및 공유에 용이
- 계산 리소스 절약

장점
- 빠른 모델 로딩
- 버전 관리 가능
- 다양한 환경에서 사용하기 용이

모델의 직렬화는 AI 개발과 배포 과정에서 중요한 단곙비니다. 효율적인 모델관리와 재사용을 가능하게 하는 방법입니다.  
모든게 질렬화가 가능하지 않기때문에 확인을 하고 직렬화를 해야 합니다. `is_lc_serializable` 를 실행하여 LnagChain 클래스가 직렬화 가능한지 확인할 수 있습니다.

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt = PromptTemplate.from_template("{coin}에 대해서 200자 내외로 요약해주세요.")

클래스(Class)에 대하여 직렬화과 가능한지 확인합니다.

In [4]:
print(f"ChatOpenAI: {ChatOpenAI.is_lc_serializable()}")

ChatOpenAI: True


LLM 객체에 대한 직렬화 가능여부도 확인할 수 있습니다.

In [5]:
print(f"llm: {llm.is_lc_serializable()}")

llm: True


chain 의 직렬화 가능 여부도 체크할 수 있습니다.

In [6]:
chain = prompt | llm
print(f"chain: {chain.is_lc_serializable()}")

chain: True


### Chain 직렬화 방법 (dumps, dumpd)

Chain을 직렬화 하는 방법과 직렬화 된 Chain을 불러오는 방법입니다.  

직렬화 방법
- `dumpd`: 객체의 속성을 dict 형태로 저장합니다.  

In [7]:
from langchain_core.load import dumpd, dumps

dumpd_chain = dumpd(chain)
dumpd_chain

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'runnable', 'RunnableSequence'],
 'kwargs': {'first': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'prompts', 'prompt', 'PromptTemplate'],
   'kwargs': {'input_variables': ['coin'],
    'template': '{coin}에 대해서 200자 내외로 요약해주세요.',
    'template_format': 'f-string'},
   'name': 'PromptTemplate'},
  'last': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'chat_models', 'openai', 'ChatOpenAI'],
   'kwargs': {'model_name': 'gpt-4o-mini',
    'temperature': 0.0,
    'openai_api_key': {'lc': 1, 'type': 'secret', 'id': ['OPENAI_API_KEY']},
    'max_retries': 2,
    'n': 1},
   'name': 'ChatOpenAI'}},
 'name': 'RunnableSequence'}

- `dumps`: 객체를 Json 속성으로 저장합니다.

In [8]:
dumps_chain = dumps(chain)
dumps_chain

'{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "runnable", "RunnableSequence"], "kwargs": {"first": {"lc": 1, "type": "constructor", "id": ["langchain", "prompts", "prompt", "PromptTemplate"], "kwargs": {"input_variables": ["coin"], "template": "{coin}\\uc5d0 \\ub300\\ud574\\uc11c 200\\uc790 \\ub0b4\\uc678\\ub85c \\uc694\\uc57d\\ud574\\uc8fc\\uc138\\uc694.", "template_format": "f-string"}, "name": "PromptTemplate"}, "last": {"lc": 1, "type": "constructor", "id": ["langchain", "chat_models", "openai", "ChatOpenAI"], "kwargs": {"model_name": "gpt-4o-mini", "temperature": 0.0, "openai_api_key": {"lc": 1, "type": "secret", "id": ["OPENAI_API_KEY"]}, "max_retries": 2, "n": 1}, "name": "ChatOpenAI"}}, "name": "RunnableSequence"}'

타입을 확인해보겠습니다.

In [9]:
print(f"dumpd_chain : {type(dumpd_chain)}")
print(f"dumps_chain : {type(dumps_chain)}")

dumpd_chain : <class 'dict'>
dumps_chain : <class 'str'>


### Pickle 파일

Pickle 파일은 Python 객체를 바이너리 형태로 직렬화 하는 포맷입니다.  
Python 객체를 빠르게 저장하고 로드할 수 있는 특별한 타입입니다.  

- `pickle.dump()`: 객체를 파일에 저장
- `pickle.load()`: 파일에서 객체 로드

pickle 파일로 저장하는 방법입니다.

In [10]:
import pickle

# dumpd_example_chain.pkl 파일로 직렬화된 체인을 저장
with open("dumpd_example_chain.pkl", "wb") as f:
    pickle.dump(dumpd_chain, f)

Json 형식으로 저장할 수도 있습니다.

In [12]:
import json

with open("dumpd_example_chain.json", "w") as f:
    json.dump(dumpd_chain, f)

### load: 저장한 모델(체인) 불러오기

pickle 파일을 로드하는 방법입니다.

In [15]:
import pickle

with open("dumpd_example_chain.pkl", "rb") as f:
    load_dumpd_chain = pickle.load(f)

In [18]:
load_dumpd_chain

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'runnable', 'RunnableSequence'],
 'kwargs': {'first': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'prompts', 'prompt', 'PromptTemplate'],
   'kwargs': {'input_variables': ['coin'],
    'template': '{coin}에 대해서 200자 내외로 요약해주세요.',
    'template_format': 'f-string'},
   'name': 'PromptTemplate'},
  'last': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'chat_models', 'openai', 'ChatOpenAI'],
   'kwargs': {'model_name': 'gpt-4o-mini',
    'temperature': 0.0,
    'openai_api_key': {'lc': 1, 'type': 'secret', 'id': ['OPENAI_API_KEY']},
    'max_retries': 2,
    'n': 1},
   'name': 'ChatOpenAI'}},
 'name': 'RunnableSequence'}

로드한 Json파일을 `load` 메서드를 활용하여 불러옵니다.

In [16]:
from langchain_core.load import load

chain_from_dumpd_file = load(load_dumpd_chain)

response = chain_from_dumpd_file.invoke({"coin": "이더리움"})
print(response.content)

이더리움은 블록체인 기반의 분산형 플랫폼으로, 스마트 계약과 분산 애플리케이션(dApp)을 개발할 수 있게 해줍니다. 2015년 비탈릭 부테린에 의해 출시되었으며, 이더(Ether, ETH)라는 암호화폐를 사용합니다. 이더리움은 탈중앙화된 금융(DeFi), NFT, DAO 등 다양한 분야에서 활용되며, 지속적으로 발전하고 있습니다. 최근에는 에너지 효율성을 높이기 위한 지분 증명(PoS) 방식으로 전환되었습니다.


Json파일을 로드하는 방법입니다.

In [19]:
import json

with open("dumpd_example_chain.json", "rb") as f:
    load_json_chain = json.load(f)

loaded_chain = load(load_json_chain)
json_response = loaded_chain.invoke({"coin": "이더리움"})
print(json_response.content)

이더리움은 블록체인 기반의 분산형 플랫폼으로, 스마트 계약과 분산 애플리케이션(dApp)을 개발할 수 있게 해줍니다. 2015년 비탈릭 부테린에 의해 출시되었으며, 이더(Ether, ETH)는 플랫폼의 기본 암호화폐입니다. 이더리움은 탈중앙화된 금융(DeFi), NFT(대체 불가능한 토큰) 등 다양한 분야에서 활용되고 있습니다. 최근에는 이더리움 2.0으로의 업그레이드가 진행 중이며, 이는 에너지 효율성을 높이고 거래 속도를 개선하는 데 중점을 두고 있습니다.
